# DOE: Features

##### Note: This notebook just generates potential experiment groups of different sizes (k) - final experiment groups were chosen manually (see excel sheet)

#### Description:

1) Read in language vectors and their dataset abbreviation

2) Remove investigated feature from the dataframe and find k closest vectors for feature

3) Create overview over all features for k

4) Generate for each k a csv-file -> experiment groups are chosen manually in excel

5) Create scripts per feature from manual choice (csv-file)

In [3]:
import pandas as pd
import datetime

#### 5) Generate experiment script files

In [7]:
# doe is the table with features as a dataframe
def df_per_feature(doe, feat_number):
    feature ="Feature "+str(feat_number)
    feat_1 = doe['FEATURE'] == feature 
    return doe[feat_1]
    

In [8]:
# from language similarity

def generate_script_file(doe):
    
#     df = dist_filter(df_raw, d)
#     print(df)
#     number_of_experiments = df['DIST'].value_counts().sort_index()[d]
    #concatenation
# false_lan_list = .. 
# true_lan_list = ..
# eval_list = ..

    for i in range(13,14):
        df = df_per_feature(doe,i)
 
        false_lan_list = df['FALSE - Experiment group'].to_list()  
        false_lan_list  = [x for x in false_lan_list  if str(x) != 'nan'] 
        
        eval_list = df['Evaluation 1'].to_list() 
        eval_list  = [x for x in eval_list  if str(x) != 'nan']
        
        true_no_eval_lan_list = df['True no eval'].to_list() 
        true_no_eval_lan_list  = [x for x in true_no_eval_lan_list  if str(x) != 'nan'] 
               
        
        file = open('../experiments/feature_'+str(i)+'/run_transformers.sh','w')
        #file = open('run_transformers.sh','w')

        file.write('# @author: jopo, phze \n')
        file.write('# @date: ' + str(datetime.datetime.now()) + '\n')
        file.write('\n')
        file.write('\n') 
        
        # do it for four experiments: 
        for experiment in range(1,5):
            file.write('\n')
            file.write('\n')
            file.write("# Experiment" +str(experiment)+"\n")
            file.write('\n') 
            file.write("cat")
            for j in false_lan_list:
                file.write(" 'languages/" + j + "/dataset_"+str(experiment)+".txt'")#some_df[i] lists languages for finetuning1 
            file.write("> 'languages/f" + str(i) +  "/dataset_false_lan.txt'\n")

            file.write("cat")
            for j in eval_list:
                file.write(" 'languages/" + j +"/dataset_"+str(experiment)+".txt'")
            file.write("> 'languages/f" + str(i) +  "/dataset_eval.txt'\n")#some_other_df[i] lists languages for finetuning2 

            file.write("cat")
            for j in true_no_eval_lan_list:
                file.write(" 'languages/" + j +"/dataset_"+str(experiment)+".txt'")
            file.write("> 'languages/f" + str(i) +  "/dataset_true_no_eval.txt'\n") 
            file.write('\n')
            file.write("# Experiment 1: fine tune 2 on all false languages. Fin tune 2 half false half true no eval. Eval on eval_lan\n") 
            file.write("# Experiment 2: fine tune 2 on all false languages. Fin tune 2 half false half lan from eval_lan in experiment 1. Eval on previous 'true_no_eval'\n") 
            # finetuning 1
            file.write("python3 preprocessing/take_sentences.py  'languages/f"+str(i)+"/dataset_false_lan.txt' 'languages/f"+str(i)+"/dataset_false_lan.txt' '17003'\n")
            # finetuning 2
            file.write("python3 preprocessing/take_sentences.py  'languages/f"+str(i)+"/dataset_false_lan.txt' 'languages/f"+str(i)+"/dataset_false_lan_half.txt' '8502'\n")
            file.write("python3 preprocessing/take_sentences.py  'languages/f"+str(i)+"/dataset_true_no_eval.txt' 'languages/f"+str(i)+"/dataset_true_lan_half_1.txt' '8502'\n")
            file.write("cat 'languages/f"+str(i)+"/dataset_false_lan_half.txt' 'languages/f"+str(i)+"/dataset_true_lan_half_1.txt' > 'languages/f" + str(i) +  "/dataset_fin_tun_2_1.txt'\n") 
            #shuffle
            file.write("python3 preprocessing/take_sentences.py 'languages/f" + str(i) +  "/dataset_fin_tun_2_1.txt' 'languages/f" + str(i) +  "/dataset_fin_tun_2_1.txt' '17003'\n")
            # testing 
            file.write("python3 preprocessing/take_sentences.py 'languages/f"+str(i)+"/dataset_eval.txt' 'languages/f"+str(i)+"/dataset_eval_1.txt' '4350'\n") 

                #for experiment 2 
            #fine_tuning_2  - dataset_false_lan_half.txt
            # false group same 
            file.write("python3 preprocessing/take_sentences.py  'languages/f"+str(i)+"/dataset_eval.txt' 'languages/f"+str(i)+"/dataset_true_lan_half_2.txt' '8502'\n")  
            file.write("cat 'languages/f"+str(i)+"/dataset_false_lan_half.txt' 'languages/f"+str(i)+"/dataset_true_lan_half_2.txt' > 'languages/f" + str(i) +  "/dataset_fin_tun_2_2.txt'\n") 
            #shuffle
            file.write("python3 preprocessing/take_sentences.py 'languages/f" + str(i) +  "/dataset_fin_tun_2_2.txt' 'languages/f" + str(i) +  "/dataset_fin_tun_2_2.txt' '17003'\n")

            # testing
            file.write("python3 preprocessing/take_sentences.py 'languages/f"+str(i)+"/dataset_true_no_eval.txt' 'languages/f"+str(i)+"/dataset_eval_2.txt' '4350'\n") 

            file.write('\n')
            # run transformers
            # exp1
             #fine tuning 1
            file.write("python3 experiments/SimpleTransformers.py 'languages/f"+str(i)+"/dataset_false_lan.txt' 'languages/f"+str(i)+"/dataset_eval_1.txt' 'results/output_f"+str(i)+"/dataset_"+str(experiment)+"/exp1/fine_tuning_1'\n")
             #fine tuning 2
            file.write("python3 experiments/SimpleTransformers.py 'languages/f"+str(i)+"/dataset_fin_tun_2_1.txt' 'languages/f"+str(i)+"/dataset_eval_1.txt' 'results/output_f"+str(i)+"/dataset_"+str(experiment)+"/exp1/fine_tuning_2'\n") 
            #exp2 -switching true language in fine_tuning_2 and evaluation 
             #fine tuning 1
            file.write("python3 experiments/SimpleTransformers.py 'languages/f"+str(i)+"/dataset_false_lan.txt' 'languages/f"+str(i)+"/dataset_eval_2.txt' 'results/output_f"+str(i)+"/dataset_"+str(experiment)+"/exp2/fine_tuning_1'\n")
             #fine tuning 2
            file.write("python3 experiments/SimpleTransformers.py 'languages/f"+str(i)+"/dataset_fin_tun_2_2.txt' 'languages/f"+str(i)+"/dataset_eval_2.txt' 'results/output_f"+str(i)+"/dataset_"+str(experiment)+"/exp2/fine_tuning_2'\n")           
            #fine tuning 2_2 is false group concatenated with evaluation 1
            file.closed

#### Execution : ___main___

In [9]:
doe = pd.read_csv("doe_features_overview_wiki.csv") # should it be relative or absolute (relative now and I want to run things from main folder. 
generate_script_file(doe)